Este notebook aplica las transformaciones del csv a sus respectivos modelos

In [25]:
import os
import pandas as pd
import trimesh
import shutil
import re

In [26]:


def apply_transformation(obj_path, transformation_matrix):
    # Cargar el archivo OBJ con textura y materiales
    mesh = trimesh.load_mesh(obj_path)
    
    # Aplicar la transformación
    mesh.apply_transform(transformation_matrix)

    return mesh

def borra_mat_tex(directorio_salida):
    directorio = os.path.dirname(directorio_salida)
    for elemento in os.listdir(directorio):
        if elemento.endswith('.png') or elemento.endswith('.mtl'):
            ruta_elemento = os.path.join(directorio, elemento)
            os.remove(ruta_elemento)
    
def copia_mat_tex(obj_path, directorio_salida):
    file_name = os.path.splitext(os.path.basename(obj_path))[0]
    directorio_padre = os.path.dirname(obj_path)

    print(file_name)
    file_name_mtl = file_name + ".mtl"
    # file_name_bmp = file_name + ".bmp"
    file_name_bmp = "material_0.bmp"
    
    # Archivo .mtl
    ruta_origen = os.path.join(directorio_padre, file_name_mtl)
    ruta_destino = os.path.join(directorio_salida, file_name_mtl)
    shutil.copy(ruta_origen, ruta_destino)

    # Archivo .bmp
    ruta_origen = os.path.join(directorio_padre, file_name_bmp)
    ruta_destino = os.path.join(directorio_salida, file_name_bmp)
    shutil.copy(ruta_origen, ruta_destino)

def main(csv_file, out_folder):
    # Crear la carpeta results si no existe
    if not os.path.exists("results"):
        os.makedirs("results")
    
    # Leer el archivo CSV
    df = pd.read_csv(csv_file, header=None)
    df.head()
    
    # Iterar sobre cada fila del DataFrame
    for index, row in df.iterrows():
        if index < 0:
            continue
        obj_path = row[0]  # Ruta relativa al archivo OBJ
        transformation_matrix = row[1:].values.reshape(4, 4)  # Matriz de transformación

        # Obtener el nombre del archivo sin extensión
        # file_name = os.path.splitext(os.path.basename(obj_path))
        
        # Aplicar la transformación al archivo OBJ
        transformed_mesh = apply_transformation(obj_path, transformation_matrix)

        directorio_salida = f"results/{out_folder}/modelo_{index}"
        
        # Crear la carpeta para guardar el modelo i
        if not os.path.exists(directorio_salida):
            os.makedirs(directorio_salida)

        # Guardar el archivo OBJ transformado
        # result_path = os.path.join(directorio_salida, f"modelo_{index}.obj")
        # transformed_mesh.export(result_path)
        # print(f"Archivo {obj_path} transformado y guardado en {result_path}")
            
        result_path = os.path.join(directorio_salida, os.path.basename(obj_path))
        transformed_mesh.export(result_path)

        # Borra materiales y texturas generadas
        borra_mat_tex(directorio_salida + f"/modelo_{index}")

        # Copia los antiguos .mtl y .bmp
        copia_mat_tex(obj_path, directorio_salida)

        print(f"Archivo {obj_path} transformado y guardado en {result_path}")


In [27]:
def procesar_carpeta(carpeta, transformation_matrix):
    patron_bmp = re.compile(r'^material_\d+\.jpg$')
    for root, _, archs in os.walk(carpeta):
        for archivo in archs:
            # Verificar si el archivo tiene la extensión .obj
            if archivo.endswith(".obj"):
                ruta_archivo = os.path.join(root, archivo)
                # Aplicar la transformación al modelo y guardar el nuevo modelo
                nuevo_modelo = apply_transformation(ruta_archivo, transformation_matrix)
                #ruta_destino = os.path.join("datasets/transformado/" + archivo)
                nuevo_modelo.export(ruta_archivo)
                
                # Iterar sobre los archivos en la carpeta
                for archivo2 in os.listdir(root):
                    ruta_archivo2 = os.path.join(root, archivo2)
                    if os.path.isfile(ruta_archivo2):
                        # Verificar si el nombre del archivo coincide con el patrón
                        if patron_bmp.match(archivo2):
                            # Eliminar el archivo
                            print("Eliminado el archivo " + ruta_archivo2)
                            os.remove(ruta_archivo2)
                        if archivo2 == "material.mtl":
                            # Eliminar el archivo
                            print("Eliminado el archivo " + ruta_archivo2)
                            os.remove(ruta_archivo2)

                print(f"Archivo {archivo} transformado y guardado en {ruta_archivo}")

In [28]:
csv_file = "matrices_transformacion.csv"
out_folder = "HeadSpace nuevo"
main(csv_file, out_folder)

131117171526
Archivo results/HeadSpace-final/modelo_1213/131117171526.obj transformado y guardado en results/HeadSpace nuevo/modelo_0/131117171526.obj
131020171933
Archivo results/HeadSpace-final/modelo_661/131020171933.obj transformado y guardado en results/HeadSpace nuevo/modelo_1/131020171933.obj
131102112105
Archivo results/HeadSpace-final/modelo_900/131102112105.obj transformado y guardado en results/HeadSpace nuevo/modelo_2/131102112105.obj
131006165343
Archivo results/HeadSpace-final/modelo_398/131006165343.obj transformado y guardado en results/HeadSpace nuevo/modelo_3/131006165343.obj
131116162124
Archivo results/HeadSpace-final/modelo_1183/131116162124.obj transformado y guardado en results/HeadSpace nuevo/modelo_4/131116162124.obj
131102153205
Archivo results/HeadSpace-final/modelo_930/131102153205.obj transformado y guardado en results/HeadSpace nuevo/modelo_5/131102153205.obj
131003131855
Archivo results/HeadSpace-final/modelo_243/131003131855.obj transformado y guardado e